# Final Project

**name: Muhammad Albayati**

**SID: 84652863**

In [137]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import meshutils
from camutils import Camera, makerotation, calibratePose, decode, triangulate
import cv2
import visutils
import selectpoints
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial
%matplotlib notebook

In [183]:
def reconstruct(prefix,threshold,camL,camR):
    """
    Simple reconstruction based on triangulating matched pairs of points
    between to view which have been encoded with a 20bit gray code.

    Parameters
    ----------
    prefix : str
      prefix for where the images are stored

    threshold : float
      decodability and color threshold

    camL,camR : Camera
      camera parameters
      
    posePrefixL : str
      prefix for which pose to load
      
    Returns
    -------
    pts2L,pts2R : 2D numpy.array (dtype=float)

    pts3 : 2D numpy.array (dtype=float)

    """

    imprefixR = prefix + "/frame_C0_"
    imprefixL = prefix + "/frame_C1_"
    
    CLh,maskLh = decode(imprefixL,0,threshold)
    CLv,maskLv = decode(imprefixL,20,threshold)
    CRh,maskRh = decode(imprefixR,0,threshold)
    CRv,maskRv = decode(imprefixR,20,threshold)
    
    #Read the color images
    
    leftBgImage = plt.imread(prefix+ "/color_C1_00.png")
    rightBgImage = plt.imread(prefix + "/color_C0_00.png")
    leftFgImage = plt.imread(prefix + "/color_C1_01.png")
    rightFgImage = plt.imread(prefix + "/color_C0_01.png")
    
    #Threshold the foreground and background
    
    objMaskL = np.abs(leftFgImage-leftBgImage)>threshold
    objMaskR = np.abs(leftFgImage-leftBgImage)>threshold
    
    #Convert the masks to greyscale
    
    objMaskL = obMaskL.astype(float) / 256
    objMaskL = np.mean(obMaskL, axis = 2)
    objMaskR = obMaskR.astype(float) / 256
    objMaskR = np.mean(obMaskR, axis = 2)

    CL = CLh + 1024*CLv
    maskL = maskLh*maskLv*obMaskL
    CR = CRh + 1024*CRv
    maskR = maskRh*maskRv*obMaskR

    h = CR.shape[0]
    w = CR.shape[1]

    subR = np.nonzero(maskR.flatten())
    subL = np.nonzero(maskL.flatten())

    CRgood = CR.flatten()[subR]
    CLgood = CL.flatten()[subL]

    _,submatchR,submatchL = np.intersect1d(CRgood,CLgood,return_indices=True)

    matchR = subR[0][submatchR]
    matchL = subL[0][submatchL]

    xx,yy = np.meshgrid(range(w),range(h))
    xx = np.reshape(xx,(-1,1))
    yy = np.reshape(yy,(-1,1))

    pts2R = np.concatenate((xx[matchR].T,yy[matchR].T),axis=0)
    pts2L = np.concatenate((xx[matchL].T,yy[matchL].T),axis=0)
    
    pts3 = triangulate(pts2L,camL,pts2R,camR)
    
    
    #Record the color of one of the images
    color = leftFgImage[pts2L[1,:], pts2L[0,:],:].T

    
    return pts2L,pts2R,pts3,color

In [187]:
def generateMesh(path, maskThreshold, camL, camR, pickleFile, blim):
    """
    Calls the reconstruct function to reconstruct 3D points, prunes the bad points
    and triangles, and creates a mesh in a .ply file.

    Parameters
    ----------
    path : str
      The path to the specified object pose.

    maskThreshold : float
      decodability and color threshold

    camL,camR : Camera
      camera parameters
      
    pickelFile : str
      The pickle file to store the reults in
      
    blim : 1D array
      Array specifiying the limits of the box pruning
      
    Returns
    -------
    None

    """    
    
    resultFile = pickleFile
    pts2L, pts2R, pts3, color = reconstruct(path, maskThreshold, camL, camR)
    
    
    #
    # drop points which are outside the bounding box
    
    goodpts = np.nonzero((pts3[0,:]>blim[0])&(pts3[0,:]<blim[1]) & \
    (pts3[1,:]>blim[2])&(pts3[1,:]<blim[3])& \
    (pts3[2,:]>blim[4])&(pts3[2,:]<blim[5])) 

    pts3 = pts3[:,goodpts[0]]
    pts2L = pts2L[:,goodpts[0]]
    pts2R = pts2R[:,goodpts[0]]
    color = color[:,goodpts[0]]

    #
    # compute initial triangulation
    
    trithresh = 0.05
    
    
    Triangles = scipy.spatial.Delaunay (pts2L.T)
    tri = Triangles.simplices

    d01 = np.sqrt(np.sum(np.power(pts3[:,tri[:,0]]-pts3[:,tri[:,1]],2),axis=0))
    d02 = np.sqrt(np.sum(np.power(pts3[:,tri[:,0]]-pts3[:,tri[:,2]],2),axis=0))
    d12 = np.sqrt(np.sum(np.power(pts3[:,tri[:,1]]-pts3[:,tri[:,2]],2),axis=0))

    goodtri = (d01<trithresh)&(d02<trithresh)&(d12<trithresh)
    
    tri = tri[goodtri,:]
    
    #Remap the indicies
    
    toKeep = np.unique(tri)
    indicesMap = np.zeros(pts3.shape[1])
    indicesMap[toKeep] = np.arange(0,toKeep.shape[0])
    indicesMap = indicesMap.astype(int)
    tri = indicesMap[tri]

    
    #Remove the unferenced indicies
    
    pts3 = pts3[:,toKeep]
    pts2L = pts2L[:,toKeep]
    pts2R = pts2R[:,toKeep]
    color = color[:,toKeep]
        
        
    #Get the pose of the camera
    
    camPose = path[14:15]
    
    data = {}
    data["pts2L" + str(camPose)] = pts2L
    data["pts2R" + str(camPose)] = pts2R
    data["pts3" + str(camPose)] = pts3
    data["triangles" + str(camPose)] = tri
    
    #Save to the pickle file
    
    fid = open(pickleFile, "wb" ) 
    pickle.dump(data,fid)
    fid.close()
    
    #Generate a .ply file with the mesh
    
    meshutils.writeply(pts3,color,tri,"./meshLab/mesh"+str(camAngle)+".ply")
    
    


In [188]:
#Get the intrinsic and extrinsic parameters of the cameras

camParams = np.load("calibration.pickle")
focalLength = (camParams["fx"] + camParams["fy"])/2
prinPoint = np.array([[camParams["cx"]], 
                     [camParams["cy"]]])

rotationMatrix = np.array([[0, 0, 0],
                          [0, 0, 0], 
                          [0, 0, 0]])

translationMatrix = np.array([[0],
                            [0],
                            [0]])

camL = Camera(focalLength, prinPoint, rotationMatrix, translationMatrix)
camR = Camera(focalLength, prinPoint, rotationMatrix, translationMatrix)

#Use the first left and right camera images

imgL = plt.imread('./calib_jpg_u/frame_C1_01.jpg')
ret, cornersL = cv2.findChessboardCorners(imgL, (8,6), None)
pts2L = cornersL.squeeze().T

imgR = plt.imread('./calib_jpg_u/frame_C0_01.jpg')
ret, cornersR = cv2.findChessboardCorners(imgR, (8,6), None)
pts2R = cornersR.squeeze().T

pts3 = np.zeros((3,6*8))
yy,xx = np.meshgrid(np.arange(8),np.arange(6))
pts3[0,:] = 2.8*xx.reshape(1,-1)
pts3[1,:] = 2.8*yy.reshape(1,-1)

params_init = np.array([0, 0, 0, 0, 0, -2])


#Calibrate the cameras

camL = calibratePose(pts3, pts2L, camL, params_init)
camR = calibratePose(pts3, pts2R, camR, params_init)

In [189]:
pickleFile = "./mesh.pickle"

boxLimits = [-100, 100, -100, 100, -100, 100]

#Generate the meshes
generateMesh("./teapot/grab_0_u", 0.02, camL, camR, pickleFile, boxLimits)
generateMesh("./teapot/grab_1_u", 0.02, camL, camR, pickleFile, boxLimits)
generateMesh("./teapot/grab_2_u", 0.02, camL, camR, pickleFile, boxLimits)
generateMesh("./teapot/grab_3_u", 0.02, camL, camR, pickleFile, boxLimits)
generateMesh("./teapot/grab_4_u", 0.02, camL, camR, pickleFile, boxLimits)
generateMesh("./teapot/grab_5_u", 0.02, camL, camR, pickleFile, boxLimits)
generateMesh("./teapot/grab_6_u", 0.02, camL, camR, pickleFile, boxLimits)

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )( 14 15 )( 16 17 )( 18 19 )

loading( 20 21 )( 22 23 )( 24 25 )( 26 27 )( 28 29 )( 30 31 )( 32 33 )( 34 35 )( 36 37 )( 38 39 )

loading( 0 1 )( 2 3 )( 4 5 )( 6 7 )( 8 9 )( 10 11 )( 12 13 )